## Bridging combat messages spreadsheet to JSON 

In [6]:
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook
#import rdflib as r
from rdflib.namespace import XSD, RDF, OWL, RDFS
from rdflib import Graph, URIRef, Namespace, BNode, Literal
import json
import re
import shortuuid 

In [7]:
in_file = 'Bridging_Combat_Messages_Breakdown_v2.xlsx'
out_file = 'Bridging_Combat_Messages_Breakdown_v2.ttl'

In [8]:
# namespaces
cm = "http://purl.org/artiamas/cm/"
CM = Namespace(cm)

In [10]:
file = load_workbook(filename = in_file)
sheet = file.active

In [11]:
# read the spreadsheet data into a JSON-like python structure
all_data = []

last_column = len(list(sheet.columns))
last_row = len(list(sheet.rows))

for row in range(2, last_row + 1):
    data = {}
    data['sequence'] = row-1
    for column in range(2, last_column + 1):
        col = get_column_letter(column)
        property = sheet[col + str(1)].value
        value = sheet[col + str(row)].value
        if row > 1:
            data[property] = value
    all_data.append(data)

In [35]:
# custom BBode function adds a prefix to a short uuid sequence
def bnode(prefix = ''):
    if prefix:
        return BNode(prefix + '_' + shortuuid.uuid()[:10])
    else:
        return BNode(shortuuid.uuid()[:10])

In [36]:
# dictionary mapping a spreadsheet column name to a RDF property URI
str2property = {'Time':CM.time, 'Agent':CM.agent, 'sequence':CM.sequence, 
                'Type':CM.messageType, 'Level':CM.level, 'Grid':CM.grid,
                'Agent':CM.agent, 'Target':CM.target, 'Alert Messages':CM.alertMessage, 'id':CM.id}

**Given a string (e.g., "B CO / 1 - 22") representing a military unit we've not seen before, we parse the string to infer the unit's type (e.g., CM.Company) and its superior units (e.g., Batallion 1; Regiment 22) nd their types, adding information to the RDG graph.  The function returns the sgent's URI**

In [38]:
def parse_agent(text, infer_types=False):
    """returns """
    global instances
    # noramlize agent references
    if not text:
        return CM.NONE
    name = text.strip()
    if not name:
        return CM.NONE
    if '/' not in name:
        # sometimes there's just the local name
        name = name + ' / 1 / 22 IN'
    name = name.replace('-','/').replace(' ','_')
    instance = URIRef(cm + name)

    if name in instances:
        # We've seen this before, so just return the instance
        return instance

    instances.add(name)
    if not infer_types:
        g.add((instance, RDF.type, CM.MilitaryUnit))
        return instance
    
    platoonid = None
    
    if re.search('^\d_\d_CO', name):
        platoonid = cm + text
        text = text[2:]
      
    units = [x.strip('_') for x in text.split('/')]
    
    unitid = cm + unit
    if 'ENG' in unit:
        unit_type = CM.EngineeringCompany
    elif 'SCT_PLT' in unit:
        unit_type = CM.ScoutPlatoon
    elif 'MORTAR_PLT' in unit:
        unit_type = CM.MortarPlatoon
    elif 'CO' in unit:
        unit_type = CM.Company
    else:
        print(f"Assuming {unit} is of type cm:Company")
        unit_type = CM.Company
        
    battalionid = cm + rest
    battalion_type = CM.Battalion

    regimentid = cm + regiment
    if 'IN' in regiment:
        regiment_type = CM.InfantryRegiment
    else:
        regiment_type = CM.Regiment
        
    # add instance data
    if platoonid:
        g.add((platoonid, RDF.type, CM.Platoon))
        g.add((platoonid, CM.unitOf, unitid))
    
    g.add((unitid, RDF.type, unit_type))
    g.add((unitid, CM.unitOf, battalionid))
    
    g.add((battalionid, RDF.type, battalion_type ))
    g.add((battalionid, CM.unitOf, regimentid))
    
    g.add((regimentid, RDF.type, regiment_type))
    
    return instance

In [49]:
def add_action(msg, msg_id):
    """ add an action to the message"""
    act = bnode('ACT')
    alert = msg["Alert Messages"]
    agent = list(g.objects(msg_id, CM.agent))[0]
    target = list(g.objects(msg_id, CM.target))[0]
    g.add((msg_id, CM.action, act))
    g.add((act, CM.alertMessage, Literal(alert)))
    if 'resupply' in alert:
        g.add((act, RDF.type, CM.Resupply))
        g.add((act, CM.recipient, target))
        if "(ammo)" in alert:
            g.add((act, CM.object, CM.AMMO))
        elif "(fuel)" in alert:
            g.add((act, CM.object, CM.FUEL))
    elif 'earned SA' in alert:
        g.add((act, RDF.type, CM.EarnedSA))
        g.add((act, CM.subject, agent))
        g.add((act, CM.object, target))
    elif 'moving to fight' in alert:
        g.add((act, RDF.type, CM.Move))
        g.add((act, CM.subject, agent))
        g.add((act, CM.toward, target))
        g.add((act, CM.reason, URIRef("attack")))
    elif re.match('attacking.*against', alert):
        g.add((act, RDF.type, CM.Attack))
        g.add((act, CM.subject, agent))
        g.add((act, CM.object, target))
    elif 'fighting' in alert:
        g.add((act, RDF.type, CM.StartFight))
        g.add((act, CM.subject, agent))
        g.add((act, CM.toward, target))
    elif re.match('attacking.*ended', alert):
        g.add((act, RDF.type, CM.EndFight))
        g.add((act, CM.subject, agent))
        g.add((act, CM.toward, target))
    elif re.match('receiving.*fire$', alert):
        g.add((act, RDF.type, CM.Attack))
        g.add((act, CM.object, agent))
    elif re.match('receiving.*fire ended', alert):
        g.add((act, RDF.type, CM.EndAttack))
        g.add((act, CM.object, agent))
    elif "not going after OPFOR" in alert:
        g.add((act, RDF.type, CM.Report))
        g.add((act, CM.subject, agent))
        g.add((act, CM.Status, CM.UNABLE ))
        g.add((act, CM.reason, Literal(alert)))
        act1 = bnode('ACT')
        g.add((act, CM.action, act1 ))
        g.add((act1, RDF.type, CM.Engage))
        g.add((act1, CM.subject, agent))
        g.add((act1, CM.object, target))
    elif "paused at Crossing Control Point" in alert:
        g.add((act, RDF.type, CM.Report))
        g.add((act, CM.Status, CM.PAUSE ))
        if "Crossing is not traversable" in alert:
            g.add((act, CM.reason, Literal("Crossing is not traversable")))
        act1 = bnode('ACT')
        g.add((act, CM.action, act1 ))
        g.add((act1, RDF.type, CM.MOVE))
        g.add((act1, CM.subject, agent))
        g.add((act1, CM.object, target))
    elif 'task' in alert or 'Task' in alert or 'Task' in msg['Alert Messages']:
        # g.add((act, RDF.type, CM.EndAttack))
        g.add((act, RDF.type, CM.TaskReport))
        g.add((act, CM.subject, agent))
        # g.add((act, CM.object, agent))
    elif 'waiting' in alert:
        g.add((act, RDF.type, CM.Report))
        g.add((act, CM.subject, agent))
        g.add((act, CM.Status, CM.WAIT ))
        if "can't attack without reinforcements" in alert:
            g.add((act, CM.reason, Literal("need reinforcements")))
        act1 = bnode('ACT')
        g.add((act, CM.action, act1 ))
        g.add((act1, RDF.type, CM.Attack))
        g.add((act1, CM.subject, agent))
        g.add((act1, CM.object, target))
    elif 'Firing has stopped'in alert:
        g.add((act, RDF.type, CM.Report))
        g.add((act, CM.subject, agent))
        g.add((act, CM.Status, CM.END ))
        act1 = bnode('ACT')
        g.add((act, CM.action, act1 ))
        g.add((act1, RDF.type, CM.Attack))
        g.add((act1, CM.object, agent))
    else:
        print("Unrecognized alert:", alert)

In [50]:
# Create the inital RDF graph 
g = Graph()
CM = Namespace('http://purl.org/artiamas/cm/')
g.bind("cm", CM, override=True)
g.bind("owl", OWL, override=True)
instances = set()
task = bnode('TASK')
g.add((task, RDF.type, CM.CrossingTask))

<Graph identifier=N7283eafeee944524ba2740ec10f34c11 (<class 'rdflib.graph.Graph'>)>

In [51]:
messages = []
actions = []
for msg in all_data:
    subj = bnode('MSG')
    messages.append(subj)
    g.add((subj, RDF.type, CM.CombatMessage))
    g.add((subj, CM.task, task))
    for prop, obj in msg.items():
        if prop not in str2property:
            print(f"Unrecognized property {prop}")
            continue
        obj = str2object(prop, obj)
        prop = str2property[prop]
        g.add((subj, prop, obj))
    add_action(msg, subj)
g.add((task, CM.firstMessage, messages[0]))
g.add((task, CM.lastMessage, messages[-1]))
for i in range(len(messages)):
     g.add((messages[i], CM.nextMessage, messages[1+1]))

Unrecognized alert: Movement is resumed
Unrecognized alert: planned battle removed.  No real targets.


In [48]:
g.serialize(destination=out_file)

<Graph identifier=N54024dd529024e02807f82cdcb62ee57 (<class 'rdflib.graph.Graph'>)>

In [34]:
print(g.serialize())

@prefix cm: <http://purl.org/artiamas/cm/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://purl.org/artiamas/cm/C_CO_/_1_/_22_IN> a cm:MilitaryUnit .

<http://purl.org/artiamas/cm/D_CO_/_1_/_22_IN> a cm:MilitaryUnit .

<http://purl.org/artiamas/cm/MORTAR_PLT_/_1_/_22_IN> a cm:MilitaryUnit .

<http://purl.org/artiamas/cm/3_1_CO_(TRK)_/_1_BN_/_241_INF_BDE_(TM)> a cm:MilitaryUnit .

<http://purl.org/artiamas/cm/50_ENG_CO_(MRBC)_/_1_/_22_IN> a cm:MilitaryUnit .

<http://purl.org/artiamas/cm/2_1_CO_(TRK)_/_1_BN_/_241_INF_BDE_(TM)> a cm:MilitaryUnit .

<http://purl.org/artiamas/cm/A_CO_/_1_/_22_IN> a cm:MilitaryUnit .

<http://purl.org/artiamas/cm/SCT_PLT_/_1_/_22_IN> a cm:MilitaryUnit .

<http://purl.org/artiamas/cm/1_1_CO_(TRK)_/_1_BN_/_241_INF_BDE_(TM)> a cm:MilitaryUnit .

<http://purl.org/artiamas/cm/B_CO_/_1_/_22_IN> a cm:MilitaryUnit .

[] a cm:CombatMessage ;
    cm:action [ a cm:EndFight ;
            cm:alertMess

In [37]:
# function mapping prop's value to a RDF property URI
def str2object (prop, value):
    """ convert a string into a object or literal """
    if type(value) in [int, float]:
        return Literal(value)
    elif prop == 'Agent':
        return parse_agent(value)
    elif prop == 'Target':
        if value and " Task " in value:
            # sometimes the Target field has the task
            g.add((task, RDFS.label, Literal(value)))
            return task
        else:
            return parse_agent(value)
    else:
        return Literal(value)
    